<a href="https://colab.research.google.com/github/ykitaguchi77/ImageProcessing/blob/master/split_dataset_for_crossvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Split datasets for cross validation
データセットを10分割し、そのうち1つをtestセットに、残りの合計をtraining+validationセットに分割するスクリプト


In [2]:
import random
import glob
import os
import shutil
import numpy as np


from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#フォルダを作成

orig_path = "/content/drive/My Drive/gravcont"
dst_path = "/content/drive/My Drive/gravcont_crossvalidation"  # フォルダ名

if not os.path.exists(dst_path):  # ディレクトリがなかったら
    os.mkdir(dst_path)  # 作成したいフォルダ名を作成
    for i in range(10):
        os.mkdir(dst_path+'/'+str(i))
        #print(i)




"""
files = glob.glob(orig_path + "/*")
print(files)
sample_files = random.sample(files, copy_num)  # 規定数のファイルを無作為に選択

for p in sample_files:  # 選択したファイルを目的フォルダにコピー
    shutil.move(p, dst_path)
    print(p)

print("process done!!")
"""

In [21]:
classes = os.listdir(orig_path)
print(classes)

#データセットを分割
data_list = os.listdir(orig_path+'/'+classes[0])
print(data_list)
print(len(data_list))

split = list(np.array_split(data_list, 10))
print(split[0])


['grav', 'cont']
['533.jpg', '1347.jpg', '4465.jpg', '1357.jpg', '6579.jpg', '1525.jpg', '6518.jpg', '2900.jpg', '6599.jpg', '1503.jpg', '4681.jpg', '5015.jpg', '5673.jpg', '2853.jpg', '4486.jpg', '2851.jpg', '2684.jpg', '4606.jpg', '2644.jpg', '2042.jpg', '429.jpg', '2960.jpg', '3759.jpg', '7077.jpg', '6147.jpg', '4008.jpg', '6965.jpg', '5890.jpg', '4713.jpg', '6855.jpg', '4805.jpg', '466.jpg', '6106.jpg', '3600.jpg', '4332.jpg', '2630.jpg', '7936.jpg', '5820.jpg', '6150.jpg', '1931.jpg', '1385.jpg', '5260.jpg', '5550.jpg', '7428.jpg', '2231.jpg', '3144.jpg', '374.jpg', '3164.jpg', '3162.jpg', '4206.jpg', '3748.jpg', '1453.jpg', '5758.jpg', '2726.jpg', '1438.jpg', '1841.jpg', '4272.jpg', '6555.jpg', '893.jpg', '7765.jpg', '7049.jpg', '1270.jpg', '3802.jpg', '1682.jpg', '1020.jpg', '465.jpg', '4331.jpg', '6764.jpg', '4030.jpg', '3444.jpg', '180.jpg', '8198.jpg', '6806.jpg', '2148.jpg', '2668.jpg', '3918.jpg', '2709.jpg', '1396.jpg', '7291.jpg', '3568.jpg', '6662.jpg', '876.jpg', '3484.

In [18]:
import numpy as np
l = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
list(np.array_split(l, 3)) # [array([1, 2, 3, 4]), array([5, 6, 7]), array([ 8,  9, 10])]


[array([1, 2, 3, 4]), array([5, 6, 7]), array([ 8,  9, 10])]

#**作ったフォルダを削除したいとき**

In [12]:
directory = '/content/drive/My Drive/gravcont_crossvalidation'

try:
    shutil.rmtree(directory)
except FileNotFoundError:
    pass